In [1]:
text = """
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
"""
lines = text.strip().split("\n")
lines

['Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green',
 'Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue',
 'Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red',
 'Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red',
 'Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green']

In [2]:
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class Bag:
    blue: Optional[int] = 0
    red: Optional[int] = 0
    green: Optional[int] = 0

    @classmethod
    def from_string(cls, s: str):
        "input should look something like: 3 blue, 4 red, 2 green"
        d = {}
        for part in s.strip().split(','):
            count, color = part.split()
            d[color] = int(count)
        return cls(**d)

    def can_contain(self, other: "Bag"):
        return self.blue >= other.blue and self.red >= other.red and self.green >= other.green

example = Bag.from_string("8 green, 6 blue, 20 red")
example

Bag(blue=6, red=20, green=8)

In [3]:
benchmark = Bag(blue=14, red=12, green=13)
benchmark.can_contain(example)

False

In [4]:
@dataclass
class Game:
    id: int
    bags: List[Bag]

    @classmethod
    def from_string(cls, s: str):
        "input sould look something like: Game 1: 3 blue; ..."
        id_part, bag_parts = s.split(':', 1)
        id = id_part.split()[-1]
        bags = []
        for part in bag_parts.split(';'):
            bag = Bag.from_string(part)
            bags.append(bag)
        return cls(id=int(id), bags=bags)

    def check(self, max_bag: Bag):
        for bag in self.bags:
            if not max_bag.can_contain(bag):
                return False
        return True

Game.from_string('Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green\n')

Game(id=5, bags=[Bag(blue=1, red=6, green=3), Bag(blue=2, red=1, green=2)])

In [5]:
games = []
for line in lines:
    game = Game.from_string(line)
    games.append(game)
games

[Game(id=1, bags=[Bag(blue=3, red=4, green=0), Bag(blue=6, red=1, green=2), Bag(blue=0, red=0, green=2)]),
 Game(id=2, bags=[Bag(blue=1, red=0, green=2), Bag(blue=4, red=1, green=3), Bag(blue=1, red=0, green=1)]),
 Game(id=3, bags=[Bag(blue=6, red=20, green=8), Bag(blue=5, red=4, green=13), Bag(blue=0, red=1, green=5)]),
 Game(id=4, bags=[Bag(blue=6, red=3, green=1), Bag(blue=0, red=6, green=3), Bag(blue=15, red=14, green=3)]),
 Game(id=5, bags=[Bag(blue=1, red=6, green=3), Bag(blue=2, red=1, green=2)])]

In [6]:
benchmark

Bag(blue=14, red=12, green=13)

In [7]:
for game in games:
    print(f"ID: {game.id}. Check: {game.check(benchmark)}")

ID: 1. Check: True
ID: 2. Check: True
ID: 3. Check: False
ID: 4. Check: False
ID: 5. Check: True


In [8]:
id_sum = 0
for game in games:
    if game.check(benchmark):
        id_sum += game.id

id_sum

8

In [9]:
%%timeit

from dataclasses import dataclass
from typing import List, Optional

@dataclass
class Bag:
    blue: Optional[int] = 0
    red: Optional[int] = 0
    green: Optional[int] = 0

    @classmethod
    def from_string(cls, s: str):
        "input should look something like: 3 blue, 4 red, 2 green"
        d = {}
        for part in s.strip().split(','):
            count, color = part.split()
            d[color] = int(count)
        return cls(**d)

    def can_contain(self, other: "Bag"):
        return self.blue >= other.blue and self.red >= other.red and self.green >= other.green

@dataclass
class Game:
    id: int
    bags: List[Bag]

    @classmethod
    def from_string(cls, s: str):
        "input sould look something like: Game 1: 3 blue; ..."
        id_part, bag_parts = s.split(':', 1)
        id = id_part.split()[-1]
        bags = []
        for part in bag_parts.split(';'):
            bag = Bag.from_string(part)
            bags.append(bag)
        return cls(id=int(id), bags=bags)

    def check(self, max_bag: Bag):
        for bag in self.bags:
            if not max_bag.can_contain(bag):
                return False
        return True


lines = open('../data/day02.txt').readlines()
benchmark = Bag(blue=14, red=12, green=13)

games = []
for line in lines:
    game = Game.from_string(line)
    games.append(game)
    
id_sum = 0
for game in games:
    if game.check(benchmark):
        id_sum += game.id

id_sum

1.12 ms ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
